In [1]:
#EJERCICIO DATASET LOGS NASA
#JORGE CASAN VÁZQUEZ

from pyspark import SparkContext #Todo lo necesario para trabajar con Spark, solo lo ejecutaremos una vez
sc = SparkContext()

In [3]:
#Tenemos que descargarlo a través de nuestro Dropbox para que excede el número de megas para descargarselo en local
import urllib.request
f = urllib.request.urlretrieve("https://www.dropbox.com/s/73wr8xb5s6fdj7g/apache.access.log.PROJECT?dl=1", "apache.access.log.PROJECT")

In [5]:
datos = "./apache.access.log.PROJECT"
datos_raw = sc.textFile(datos) #Con esta función nos crearemos el RDD

In [6]:
datos_raw.count() #Como si de un analisis exploratorio de datos se tratara

1043177

In [7]:
datos_raw.take(5) #Escogemos las 5 primeras observaciones

['in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839',
 'uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/USA-logosmall.gif HTTP/1.0" 304 0']

In [9]:
datos_tratados = datos_raw.map(lambda x: x.replace(" -", ' 0')).map(lambda x: x.split(" "))
#Es muy importante reemplazar los guiones por 0, de cara a realizar cálculos numéricos ya que los strings no son operables.
#Lo primero que tendremos que hacer es el replace y luego con el split ya que no podremos hacer un replace porque se convierte
#a lista, seguidamente con el split conseguimos dividir el total de las observaciones (separar las filas para poder operar)
#Con el parseamiento de los datos los tratamos para que tengan el formato adecuado

In [10]:
#Vamos a recorrer los datos imprimiendo cada columna para ver si está todo correcto
for i in range(0,10):
    datos_test = datos_tratados.map(lambda x: x[i])
    print(datos_test.take(1))
#Tenemos 9 columnas, de la 0 hasta la 9, inclusive, y para poder tratar los datos tenemos que desmenuzar cada fila
#por cada categoría, esto es, columnas del RDD

['in24.inetnebr.com']
['0']
['0']
['[01/Aug/1995:00:00:01']
['00400]']
['"GET']
['/shuttle/missions/sts-68/news/sts-68-mcc-05.txt']
['HTTP/1.0"']
['200']
['1839']


In [14]:
#### EJERCICIO 1 ######
#Calcular el mínimo, máximo y media del tamaño de las peticiones (size)
#Podemos transformar los datos aplicando map al dataset para obtener solo la columna deseada

datos_split = datos_tratados.map(lambda x: int(x[-1])) #Accederemos a la última columna (size) y lo trataremos como un integer
#para poder realizar los siguientes cálculos numéricos
#En vez de que me devuelva los resultados sólo como números, prefiero que me haga un print con lo que quiero que me extraiga
print("Hay un maximo de {} peticiones".format(datos_split.max()))
print("Hay un minimo de {} peticiones".format(datos_split.min()))
print("Hay una media de {} peticiones".format(round(datos_split.mean(),3))) #Le digo que me lo redondeee a 3 decimales

Hay un maximo de 3421948 peticiones
Hay un minimo de 0 peticiones
Hay una media de 17531.556 peticiones


In [16]:
####### EJERCICIO 2 #########
### Numero de peticiones de cada codigo de respuesta (response_code)

from operator import add #Esta es la funcion de agregación, tenemos que importarla para poder hacer el reduceByKey(add)
response_code = datos_tratados.map(lambda y: (int(y[-2]), 1)) #1 es un integer que lo que va a hacer es un conteo de cada valor
# y partir de aquí contaremos cuantos códigos de respuesta existen por cada fila
response_code.take(10)

[(200, 1),
 (304, 1),
 (304, 1),
 (304, 1),
 (304, 1),
 (200, 1),
 (304, 1),
 (200, 1),
 (200, 1),
 (200, 1)]

In [18]:
#la Key es el response_code y lo que hacemos es agregar cuántas existen y las juntamos a través de la función de agregación
#SortByKey te lo ordena, por orden descendente o ascendente, en este caso, tendremos que especificarselo, ya que por defecto
#te las ordena en forma ascendente
#A través del comando collect () te muestra el resultado,  básicamente es una acción que te devuelve el resultado
response_code.reduceByKey(add).sortByKey(False).collect()

[(501, 17),
 (500, 2),
 (404, 6185),
 (403, 58),
 (304, 79824),
 (302, 16244),
 (200, 940847)]

In [19]:
##### EJERCICIO 3 ######
# Mostrar 20 hosts que han sido visitados mas de 10 veces
hosts = datos_tratados.map(lambda y: (y[0], 1))
agrupados = hosts.reduceByKey(add).sortByKey().filter(lambda x: x[1] > 10).take(20)
agrupados
# Key son los host y realizando el conteo por cada fila los agrupamos y a través del filter seleccionamos aquellos que han sido
#visitados más de 10 veces y finalmente mostraremos 20 de ellos

[('***.novo.dk', 16),
 ('03-dynamic-c.wokingham.luna.net', 39),
 ('05-dynamic-c.wokingham.luna.net', 25),
 ('0875pr3e.pro.rpslmc.edu', 16),
 ('1.ts1.mnet.medstroms.se', 32),
 ('10.ts2.mnet.medstroms.se', 15),
 ('101.irri.cgiar.org', 17),
 ('1032015.ksc.nasa.gov', 90),
 ('10forward.ppp.america.com', 18),
 ('10md423.uni-duisburg.de', 39),
 ('118.32.med.umich.edu', 15),
 ('12.ts2.mnet.medstroms.se', 25),
 ('120.33.med.umich.edu', 14),
 ('128.100.180.37', 15),
 ('128.100.183.252', 13),
 ('128.100.183.253', 36),
 ('128.100.191.43', 31),
 ('128.100.197.90', 16),
 ('128.100.8.177', 40),
 ('128.101.13.25', 15)]

In [20]:
##### EJERCICIO 4 #####
# Mostrar los 10 endpoints mas visitados
#Un endpoint es el último sitio que el usuario ha accedido, la cual se localiza en la columna 6, de nuevo tendremos que realizar 
#el conteo, siendo la Key el endpoint. En este caso no pondremos la sortbykey, solamente es sortby porque queremos los 10
#endpoints más visitados
endpoint = datos_tratados.map(lambda y: (y[6], 1))
endpoint_agr = endpoint.reduceByKey(add).sortBy(lambda x: x[1], False).take(10)
endpoint_agr

[('/images/NASA-logosmall.gif', 59666),
 ('/images/KSC-logosmall.gif', 50420),
 ('/images/MOSAIC-logosmall.gif', 43831),
 ('/images/USA-logosmall.gif', 43604),
 ('/images/WORLD-logosmall.gif', 43217),
 ('/images/ksclogo-medium.gif', 41267),
 ('/ksc.html', 28536),
 ('/history/apollo/images/apollo-logo1.gif', 26766),
 ('/images/launch-logo.gif', 24742),
 ('/', 20175)]

In [21]:
##### EJERCICIO 5 ######
#Mostrar los endpoints más visitados que no tienen código de respuesta igual a 200
#La columna 6 son los endpoints y la penúltima columna son los códigos de respuesta
#Tenemos que acceder a las columnas endpoint y response code para poder realizar posteriormente el filtrado
endpointNO = datos_tratados.map(lambda y: (y[6], y[-2]))
endpointNO_agr = endpointNO.sortBy(lambda x: x[1], False).filter(lambda x: x[1] != 200).take(10)
# A través del x[1] accederemos a la columna responde code y a partir de aquí hacemos el filtrado de aquellas que no contengan el
#código de respuesta 200
endpointNO_agr

[('/ksc.html', '501'),
 ('/ksc.html', '501'),
 ('/ksc.html', '501'),
 ('/ksc.html', '501'),
 ('/ksc.html', '501'),
 ('/images/getstats.gif', '501'),
 ('/ksc.html', '501'),
 ('/shuttle/missions/51-i/51-i-patch-small.gif', '501'),
 ('/shuttle/missions/sts-45/sts-45-patch-small.gif', '501'),
 ('/shuttle/missions/sts-6/sts-6-patch-small.gif', '501')]

In [33]:
##### EJERCICIO 6 ######
# Calcular el numero de hosts distintos
#Es relativamente sencillo el comando, ya que primero seleccionamos la columna de los hosts y luego le decimos que nos muestre
#los que son distintos para finalmente realizar el conteo
hosts_distinct = datos_tratados.map(lambda y: y[0]).distinct().count()
#Muestra un integer con el numero de hosts distintos
hosts_distinct

54507

In [ ]:
#Tendremos que importarnos las siguientes librerías para poder trabajar con SQL
from pyspark.sql import Row, SQLContext
from pyspark.sql.functions import *
sqlContext = SQLContext(sc)

In [27]:
##### EJERCICIO 7 #####
# Calcular el numero de host unicos cada dia

row_data = datos_tratados.map(lambda p: Row(
    host = p[0], 
    empty_first = int(p[1]),
    empty_second = p[2],
    date = p[3],
    protocol = p[4],
    endpoint = p[6],
    version = p[7],
    status_Code = int(p[-2]),
    size = p[-1]
    )
)

interactions_df = sqlContext.createDataFrame(row_data)

newdf = interactions_df.withColumn('date', regexp_replace('date', '\[', ''))
#Con este comando lo que pretendemos es deshacernos del corchete, ya que no aporta valor a la información de la fecha
#['[01/Aug/1995:00:00:01']
newdf2 = newdf.withColumn('date', regexp_replace('date', 'Aug', '08'))
#Con el replace lo que hacemos es sustuir el valor string August, en la columna date, como valor numérico para poder
#realizar cálculos. En este caso nuestra BBDD solo contiene datos para Agosto, si hubieramos tenido más meses del año tendríamos
#que haber realizado la misma operación para el resto de meses
newdf3 = newdf2.select(from_unixtime(unix_timestamp('date', 'dd/MM/yyyy:hh:mm:ss')).alias('date'), 'host')
#Con esta función extraemos el formato fecha y seleccionamos a su vez la columna host
newdf4 = newdf3.na.fill(0)
#Por si acaso rellenaremos los valores que contengan NA´s con valor 0.

#Con la función dayofmonth extrae de la columna fecha el día del mes que nos encontramos y a través de la función de agregación
#ordenaremos los host distintos por los días del mes
newdf4.groupby(dayofmonth("date").alias("Dia del mes")).agg(countDistinct('host').alias('Host distintos')).sort('Dia del mes')        .show()


+-----------+--------------+
|Dia del mes|Host distintos|
+-----------+--------------+
|          1|          2582|
|          3|          3222|
|          4|          4190|
|          5|          2502|
|          6|          2537|
|          7|          4106|
|          8|          4406|
|          9|          4317|
|         10|          4523|
|         11|          4346|
|         12|          2864|
|         13|          2650|
|         14|          4454|
|         15|          4214|
|         16|          4340|
|         17|          4385|
|         18|          4168|
|         19|          2550|
|         20|          2560|
|         21|          4134|
+-----------+--------------+
only showing top 20 rows



In [30]:
##### EJERCICIO 8 #####
# Calcular la media de peticiones diarias por host

parsed_df2 = newdf4.withColumn("month", month(col("date"))).withColumn("DayOfmonth", dayofmonth(col("date")))
parsed_df2.createOrReplaceTempView('parsed_df2')
#Tendremos que hacer una query en un dataframe y para ello tendremos que realizar el comando tempView
sqlContext.sql("SELECT DATE(date) Date, COUNT(host)/COUNT(DISTINCT host)AS Peticiones_diarias_host FROM parsed_df2 GROUP BY  DATE(date) ORDER BY DATE(date) ASC").show(n = 10)


+----------+-----------------------+
|      Date|Peticiones_diarias_host|
+----------+-----------------------+
|1995-08-01|     13.166537567776917|
|1995-08-03|     12.845127250155183|
|1995-08-04|     14.213365155131266|
|1995-08-05|     12.745003996802557|
|1995-08-06|     12.777296018919984|
|1995-08-07|     13.968582562104238|
|1995-08-08|     13.650022696323196|
|1995-08-09|      14.00440120454019|
|1995-08-10|     13.540791510059695|
|1995-08-11|     14.091578462954441|
+----------+-----------------------+
only showing top 10 rows



In [22]:
##### EJERCICIO 9 #####
# Mostrar una lista de 40 endpoints distintos que generan código de respuesta 404
endpoint_error = datos_tratados.map(lambda y: (y[6], int(y[-2])))
endpointerror_agr = endpoint_error.reduceByKey(add).sortBy(lambda x: x[1], False).filter(lambda x: x[1] == 404).distinct().take(40)
endpointerror_agr
#Tendremos que seleccionar la columna endpoint y el código de respuesta.La Key es el endpoint y a partir de aquí realizaremos
#el filtrado por el código de respuesta, por orden descendente y seleccionar aquellos que sean iguales a 404 y finalmente
#seleccionaremos 40 de ellos a través del comando take

[('/yahoo.com', 404),
 ('/search-page.html', 404),
 ('/rlvhp.htm', 404),
 ('/shuttle/technology/images/sts-comm.jpg', 404),
 ('/history/saturn/', 404),
 ('/shuttle/missio', 404),
 ('/magazine/p08aug/08pet14.gif', 404),
 ('/count69.gif', 404),
 ('/iu.net', 404),
 ('/software/winvn/winvn.', 404),
 ('/ico', 404),
 ('/ksc.html%3A', 404),
 ('/astronaut-candidates/pub/FTPfiles/netsites.ofinterest', 404),
 ('/shuttle/missions/sts-77/mission-sts-76.html', 404),
 ('/images/rss.images', 404),
 ('/shuttle/technology/sts-newsref/sts-toc.html', 404),
 ('/history/apollo/apollo-17/images/index.gif', 404),
 ('/de/dm/dm-asd/', 404),
 ('/missions/sts-70/imiages/', 404),
 ('/shuttle/missions/sts-45/sts-45-patch-small.gif"', 404),
 ('/byline.gif', 404),
 ('/shuttle/missions/STS-69/', 404),
 ('/shuttle/video/o-ring.jpeg', 404),
 ('/history/apollo/icons/host.gif', 404),
 ('/shuttle/missions/sts-1/sts-1-mission.html', 404),
 ('/technology/', 404),
 ('/shuttle/sts-1/sts-1-pa.jpg', 404),
 ('/shuttle/missions/m

In [23]:
##### EJERCICIO 10 #####
#Mostrar el top 25 de endpoints que más códigos de respuesta 404 generan
#Lo primero de todo tendremos que seleccionar los códigos de respuesta que sean iguales a 404y a continuación  realizaremos 
#la agregación por los endpoints que sean de los 25 primeros a través de la función sort, realizando previamente el conteo y
#escogeremos los 25 primeros de todos ellos
endp_filtrado = datos_tratados.filter(lambda x: int(x[-2]) == 404)

endpoint_top_error = endp_filtrado.map(lambda x: (x[6], 1))
endpointtoperror_agr = endpoint_top_error.reduceByKey(add).sortBy(lambda x: x[1], False).take(25)
endpointtoperror_agr

[('/pub/winvn/readme.txt', 633),
 ('/pub/winvn/release.txt', 494),
 ('/shuttle/missions/STS-69/mission-STS-69.html', 430),
 ('/images/nasa-logo.gif', 319),
 ('/elv/DELTA/uncons.htm', 178),
 ('/shuttle/missions/sts-68/ksc-upclose.gif', 154),
 ('/history/apollo/sa-1/sa-1-patch-small.gif', 146),
 ('/images/crawlerway-logo.gif', 120),
 ('/://spacelink.msfc.nasa.gov', 117),
 ('/history/apollo/pad-abort-test-1/pad-abort-test-1-patch-small.gif', 100),
 ('/history/apollo/a-001/a-001-patch-small.gif', 97),
 ('/images/Nasa-logo.gif', 85),
 ('/shuttle/resources/orbiters/atlantis.gif', 63),
 ('/history/apollo/images/little-joe.jpg', 62),
 ('/images/lf-logo.gif', 59),
 ('/shuttle/resources/orbiters/discovery.gif', 56),
 ('/shuttle/resources/orbiters/challenger.gif', 54),
 ('/robots.txt', 53),
 ('/elv/new01.gif>', 43),
 ('/history/apollo/pad-abort-test-2/pad-abort-test-2-patch-small.gif', 38),
 ('/pub/', 36),
 ('/pub', 36),
 ('/history/apollo/sa-2/sa-2-patch-small.gif', 35),
 ('/history/apollo/sa-5/

In [29]:
##### EJERCICIO 11 #####
# Top 5 días que generaron código de respuesta 404
# En primer lugar tendremos que seleccionar el campo fecha y el código de respuesta e igualar este parámetro al valor 404
#Seguidamente tendremos que ordenarlo en orden descendente y seleccionaremos el top 5
#En este ejercicico hemos tenido que trabajar con un dataframe
#Por ello que los comandos a la hora de contar y ordenar los datos varían con respecto al resto de ejercicios
newdf5 = newdf2.select(from_unixtime(unix_timestamp('date', 'dd/MM/yyyy:hh:mm:ss')).alias('date'), 'status_Code')

newdf5.filter(newdf5['status_Code'] == 404).groupBy(dayofmonth('date')).count().sort(col("count").desc()).show(n = 5)


+----------------+-----+
|dayofmonth(date)|count|
+----------------+-----+
|               7|  532|
|               8|  381|
|               6|  372|
|               4|  346|
|              15|  326|
+----------------+-----+
only showing top 5 rows

